In [ ]:
# ======================================
# 02_train_models.ipynb
# Entrena y guarda modelos por ticker usando
# los datasets procesados en data/processed/.
# ======================================

import os
import glob
import pandas as pd
from src.utils import load_config, ensure_dirs
from src.model_train import train_model_for_ticker

cfg = load_config()
ensure_dirs()

feature_cols = cfg["features"]["include"]
test_ratio = cfg["model"]["test_split_ratio"]

print("=== FEATURES USADAS POR EL MODELO ===")
print(feature_cols)

print("\nBuscando datasets en data/processed/ ...")
processed_files = glob.glob("data/processed/*_processed.csv")
print(processed_files)

if not processed_files:
    raise RuntimeError("No hay archivos procesados. Ejecuta primero 01_download_and_prepare.ipynb")

os.makedirs("models", exist_ok=True)

model_reports = {}

for file_path in processed_files:
    # Extraer ticker del nombre del archivo, ej: data/processed/NVDA_processed.csv -> NVDA
    base = os.path.basename(file_path)
    ticker = base.replace("_processed.csv", "")

    print(f"\nEntrenando modelo para {ticker} usando {file_path} ...")

    df = pd.read_csv(file_path, index_col=0)

    # Entrenar y guardar modelo
    model, report = train_model_for_ticker(
        df=df,
        feature_cols=feature_cols,
        model_params=cfg["model"],
        test_ratio=test_ratio,
        save_path="models",
        ticker_name=ticker
    )

    model_reports[ticker] = report

print("\n=== RESUMEN DE CLASIFICATION_REPORT POR TICKER ===")
for t, rep in model_reports.items():
    print(f"\n[{t}]")
    print(rep)

print("\nListo. Modelos guardados en carpeta models/. Puedes continuar con 03_backtest_and_analysis.ipynb o correr run_backtest.py.")
